In [2]:
import os
!pip install wget
import wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=1e396beabe6983434fcd0422a3edc55365e9d566d278e883f934fc618be9549d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [3]:
print("downloading dataset")

url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'


if not os.path.exists('./aclImdb_v1.tar.gz'):
    wget.download(url, './aclImdb_v1.tar.gz')

downloading dataset


In [4]:
!tar -xf aclImdb_v1.tar.gz

In [5]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [6]:
print(len(train_texts))
print(len(train_labels))
print(len(test_texts))
print(len(test_labels))

25000
25000
25000
25000


In [7]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [40]:
type(train_texts)
type(train_labels)

list

In [8]:
print(len(train_texts))
print(len(val_texts))

20000
5000


In [9]:
!pip install transformers torch torchvision torchaudio

     |████████████████████████████████| 2.2MB 19.1MB/s 
     |████████████████████████████████| 1.9MB 50.7MB/s 
     |████████████████████████████████| 870kB 51.4MB/s 
     |████████████████████████████████| 3.3MB 52.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=88d1e7bbeaff0b9c3bbf7b1a1d451874efac7ce3ec11b70ccc17d68811188dca
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [10]:
!nvidia-smi

Thu Apr  8 04:33:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
import torch


if torch.cuda.is_available():
    print("True")
    device = torch.device("cuda")
    
    print(torch.cuda.device_count())
    print(torch.cuda.get_device_name() )
else:
    print("False")
    device = torch.device("cpu")

True
1
Tesla T4


In [12]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [13]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [14]:
train_encodings   

In [15]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [16]:
train_dataset

In [17]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
    save_steps = 250,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss
250,0.463800
500,0.316100
750,0.259500
1000,0.256300
1250,0.234700


TrainOutput(global_step=1250, training_loss=0.3060924041748047, metrics={'train_runtime': 1052.224, 'train_samples_per_second': 1.188, 'total_flos': 4113715814400000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 2498961408, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -84557824, 'train_mem_gpu_alloc_delta': 804118016, 'train_mem_cpu_peaked_delta': 193105920, 'train_mem_gpu_peaked_delta': 6540809216})

In [18]:
trainer.evaluate(
    eval_dataset = val_dataset
    
)

{'epoch': 1.0,
 'eval_loss': 0.2032819241285324,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 2718475776,
 'eval_runtime': 102.083,
 'eval_samples_per_second': 48.98}

In [20]:
predictions = trainer.predict(test_dataset=test_dataset)

In [24]:
predictions

PredictionOutput(predictions=array([[-2.6518784,  2.5239518],
       [-2.0802402,  1.9276981],
       [-2.1844354,  2.039296 ],
       ...,
       [ 1.1535   , -1.3469938],
       [ 1.700098 , -1.9089317],
       [ 2.2581382, -2.525582 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.19716207683086395, 'test_runtime': 506.8845, 'test_samples_per_second': 49.321, 'test_mem_cpu_alloc_delta': 0, 'test_mem_gpu_alloc_delta': 0, 'test_mem_cpu_peaked_delta': 0, 'test_mem_gpu_peaked_delta': 2517502976})

In [28]:
predictions_arr = predictions[0].tolist()

In [ ]:
predictions_arr

In [ ]:
import numpy as np


In [35]:
flat_predictions = np.argmax(predictions_arr, axis=1).flatten()
flat_predictions

array([1, 1, 1, ..., 0, 0, 0])

In [36]:
len(flat_predictions)

25000

In [37]:
len(predictions_arr)

25000

In [ ]:
!rm -rf results
!rm -rf logs


In [ ]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(1):
    print("starting epoch {}...".format(epoch))
    count = 0
    for batch in train_loader:
        # print("batch ", count)
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        count = count+1

print('finished')



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

starting epoch 0...
finished


In [ ]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
os.mkdir('./results/v2')
output_model_file = os.path.join('./results/v2/', "pytorch_model.bin")
torch.save(model_to_save.state_dict(), output_model_file)

In [ ]:
model.eval()

eval_dataloader = DataLoader(val_dataset, batch_size=64, shuffle = True)

for epoch in range(1):
    print("starting epoch {}...".format(epoch))
    for batch in eval_dataloader:
        print("starting batch {}..".format(batch))
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()